# Day 2  进阶作业-HH model  

In this section, we try to understand how to build conductance-based biophysical neuron models.  

Please follow the comment instruction to fill the *todo* blanks of the following code. Then run all the cells to see the simulation result.

In [17]:
import numpy as np

import brainpy as bp
import brainpy.math as bm

## ``master_type`` organizes structures between neurons and ion channels 

``master_type`` determines what information will be passed into ``.reset_state()`` and ``update()`` function in a model.

In [18]:
class IK(bp.dyn.IonChannel):
  master_type = bp.dyn.CondNeuGroup
  
  def update(self, V, *args, **kwargs):
    pass
  
  def reset_state(self, V, *args, **kwargs):
    pass

For the above ``IK`` model, its ``master_type: bp.dyn.CondNeuGroup`` will give ``V`` variable into this node. Therefore, ``IK`` model can utilize ``V`` to update or reset its states. 

In [19]:
class ICa(bp.dyn.IonChannel):
  master_type = bp.dyn.Calcium
  
  def update(self, V, C, E, *args, **kwargs):
    pass
  
  def reset_state(self, V, C, E, *args, **kwargs):
    pass

For ``ICa`` class, its ``master_type (bp.dyn.Calcium)`` will deliver the concentration of Calcium ``C`` and the reversal potential of Calcium ion ``E`` into this node. Moreover, since the ``master_type`` of ``bp.dyn.Calcium`` is ``bp.dyn.CondNeuGroup``, it will inherit the passing of ``bp.dyn.CondNeuGroup`` and deliver ``V`` into ``ICa`` class too. 

In [20]:
class ICaNa(bp.dyn.IonChannel):
  master_type = bp.mixin.JointType[bp.dyn.Calcium, bp.dyn.Sodium]
  
  def update(self, V, Ca_info, Na_info, *args, **kwargs):
    pass
  
  def reset_state(self, V, Ca_info, Na_info, *args, **kwargs):
    pass

If an ion channel depends on more than two ion types, it can define ``master_type`` as a joint type by using ``brainpy.mixin.JointType``. For example, the above ``ICaNa`` class depends on ``bp.dyn.Calcium`` and ``bp.dyn.Sodium``, so the ``update()`` and ``reset_state()`` function depends on information of both subclasses and their parents. 

For an existing ion channel, users can check the ``master_type`` using:

In [21]:
bp.dyn.INa_Ba2002v2.master_type

brainpy._src.dyn.ions.sodium.Sodium

In [22]:
bp.dyn.INa_Ba2002.master_type

brainpy._src.dyn.neurons.hh.HHTypedNeuron

## Build a HH model by composing existing ion channels

Here is an example by building a HH neuron model by composing existing ion channels. 

In [23]:
class HH(bp.dyn.CondNeuGroupLTC):
  def __init__(self, size):
    super().__init__(size)

    self.INa = bp.dyn.INa_HH1952(size)
    self.IK = bp.dyn.IK_HH1952(size)
    self.IL = bp.dyn.IL(size, E=-54.387, g_max=0.03)

In [24]:
hh = HH(1)

runner = bp.DSRunner(hh, monitors={'na-p': hh.INa.p, 'na-q': hh.INa.q, 'k-p': hh.IK.p, 'v': hh.V})

inputs = np.ones(1000) * 4.
_ = runner.run(inputs=inputs)


  0%|          | 0/1000 [00:00<?, ?it/s]

In [25]:
bp.visualize.line_plot(runner.mon.ts, runner.mon['na-p'], legend='Na-p')
bp.visualize.line_plot(runner.mon.ts, runner.mon['na-q'], legend='Na-q')
bp.visualize.line_plot(runner.mon.ts, runner.mon['k-p'], legend='K-p', show=True)

<Figure size 432x288 with 1 Axes>

In [26]:
bp.visualize.line_plot(runner.mon.ts, runner.mon['v'], show=True)

<Figure size 432x288 with 1 Axes>

## Customizing ion channels

To customize an ion channel that can be composed using the above interface, users should define a normal ``DynamicalSystem`` with the specification of ``master_type``.  

Here are several examples:

For a potassium ion channel:  

$$  
\begin{split}\begin{aligned}  
 I_{\mathrm{K}} &= g_{\mathrm{max}} * p^4 \\  
 \frac{dp}{dt} &= \phi * (\alpha_p (1-p) - \beta_p p) \\  
 \alpha_{p} &= \frac{0.01 (V -V_{sh} + 10)}{1-\exp \left(-\left(V-V_{sh}+ 10\right) / 10\right)} \\  
 \beta_p &= 0.125 \exp \left(-\left(V-V_{sh}+20\right) / 80\right)  
 \end{aligned}\end{split}  
$$  

where $V_{sh}$ is the membrane shift (default -45 mV), and $\phi$  is the temperature-dependent factor (default 1.).

In [27]:
class IK(bp.dyn.IonChannel):
  master_type = bp.dyn.HHTypedNeuron
  
  def __init__(self, size, E=-77., g_max=36., phi=1., method='exp_auto'):
    super().__init__(size)
    self.g_max = g_max
    self.E = E
    self.phi = phi

    self.integral = bp.odeint(self.dn, method=method)

  def dn(self, n, t, V):
    alpha_n = 0.01 * (V + 55) / (1 - bm.exp(-(V + 55) / 10))
    beta_n = 0.125 * bm.exp(-(V + 65) / 80)
    return self.phi * (alpha_n * (1. - n) - beta_n * n)
  
  def reset_state(self, V, batch_or_mode=None, **kwargs):
    self.n = bp.init.variable_(bm.zeros, self.num, batch_or_mode)
  
  def update(self, V):
    t = bp.share.load('t')
    dt = bp.share.load('dt')
    self.n.value = self.integral(self.n, t, V, dt=dt)

  def current(self, V):
    return self.g_max * self.n ** 4 * (self.E - V)

For a sodium ion channel,  

$$  
\begin{split}\begin{split}  
\begin{aligned}  
   I_{\mathrm{Na}} &= g_{\mathrm{max}} m^3 h \\  
   \frac {dm} {dt} &= \phi (\alpha_m (1-x)  - \beta_m) \\  
   &\alpha_m(V) = \frac {0.1(V-V_{sh}-5)}{1-\exp(\frac{-(V -V_{sh} -5)} {10})}  \\  
   &\beta_m(V) = 4.0 \exp(\frac{-(V -V_{sh}+ 20)} {18})  \\  
   \frac {dh} {dt} &= \phi (\alpha_h (1-x)  - \beta_h) \\  
   &\alpha_h(V) = 0.07 \exp(\frac{-(V-V_{sh}+20)}{20})  \\  
   &\beta_h(V) = \frac 1 {1 + \exp(\frac{-(V -V_{sh}-10)} {10})} \\  
\end{aligned}  
\end{split}\end{split}  
$$  

where $V_{sh}$ is the membrane shift (default -45 mV), and $\phi$  is the temperature-dependent factor (default 1.).

In [28]:
class INa(bp.dyn.IonChannel):
  master_type = bp.dyn.HHTypedNeuron
  
  def __init__(self, size, E= 50., g_max=120., phi=1., method='exp_auto'):
    super(INa, self).__init__(size)
    self.g_max = g_max
    self.E = E
    self.phi = phi
    self.integral = bp.odeint(bp.JointEq(self.dm, self.dh), method=method)

  def dm(self, m, t, V):
    alpha_m = 0.11 * (V + 40) / (1 - bm.exp (-(V + 40) / 10))
    beta_m = 4* bm.exp(-(V + 65) /18)
    return self.phi * (alpha_m * (1. - m) - beta_m * m)
  
  def dh(self, h, t, V):
    alpha_h = 0.07 * bm.exp(-(V + 65) / 20)
    beta_h = 1. / (1 + bm.exp(-(V + 35) / 10))
    return self.phi * (alpha_h * (1. - h) - beta_h * h)
  
  def reset_state(self, V, batch_or_mode=None, **kwargs):
    self.m = bp.init.variable_(bm.zeros, self.num, batch_or_mode)
    self.h = bp.init.variable_(bm.zeros, self.num, batch_or_mode)
  
  def update(self, V):
    t = bp.share.load('t')
    dt = bp.share.load('dt')
    self.m.value, self.h.value = self.integral(self.m, self.h, t, V, dt=dt)

  def current(self, V):
    return self.g_max * self.m ** 3 * self.h * (self.E - V)

The leakage channel current.

In [29]:
class IL(bp.dyn.IonChannel):
  master_type = bp.dyn.HHTypedNeuron
  
  def __init__(self, size, E=-54.39, g_max=0.03):
    super(IL, self).__init__(size)
    self.g_max = g_max
    self.E = E
    
  def reset_state(self, *args, **kwargs):
    pass
  
  def update(self, V):
    pass
  
  def current(self, V):
    return self.g_max * (self.E - V)

In [30]:
class HH(bp.dyn.CondNeuGroup):
  def __init__(self, size):
    super().__init__(size, V_initializer=bp.init.Uniform(-80, -60.))
    # TODO: 初始化三个离子通道
    self.IK = ... # 参数：E=-77., g_max=36.
    self.INa = ... # 参数：E=50., g_max=120.
    self.IL = ... # 参数：E=-54.39, g_max=0.03

In [31]:
neu = HH(1)
neu.reset()

inputs = np.ones(int(200 / bm.dt)) * 1.698  # 200 ms
runner = bp.DSRunner(neu, monitors=['V', 'IK.n', 'INa.m', 'INa.h'])
runner.run(inputs=inputs)  # the running time is 200 ms

import matplotlib.pyplot as plt

plt.plot(runner.mon['ts'], runner.mon['V'])
plt.xlabel('t (ms)')
plt.ylabel('V (mV)')
plt.savefig("HH.jpg")
plt.show()

plt.figure(figsize=(6, 2))
plt.plot(runner.mon['ts'], runner.mon['IK.n'], label='n')
plt.plot(runner.mon['ts'], runner.mon['INa.m'], label='m')
plt.plot(runner.mon['ts'], runner.mon['INa.h'], label='h')
plt.xlabel('t (ms)')
plt.legend()

plt.show()

  0%|          | 0/2000 [00:00<?, ?it/s]

<Figure size 432x288 with 1 Axes>

<Figure size 432x144 with 1 Axes>

## Answer

In [32]:
import brainpy as bp
import brainpy.math as bm

  
class HH(bp.dyn.CondNeuGroup):
  def __init__(self, size):
    super().__init__(size, V_initializer=bp.init.Uniform(-80, -60.))
    self.IK = IK(size, E=-77., g_max=36.)
    self.INa = INa(size, E=50., g_max=120.)
    self.IL = IL(size, E=-54.39, g_max=0.03)

neu = HH(1)
neu.reset()

inputs = np.ones(int(200 / bm.dt)) * 1.698  # 200 ms
runner = bp.DSRunner(neu, monitors=['V', 'IK.n', 'INa.m', 'INa.h'])
runner.run(inputs=inputs)  # the running time is 200 ms

import matplotlib.pyplot as plt

plt.plot(runner.mon['ts'], runner.mon['V'])
plt.xlabel('t (ms)')
plt.ylabel('V (mV)')
plt.savefig("HH.jpg")
plt.show()

plt.figure(figsize=(6, 2))
plt.plot(runner.mon['ts'], runner.mon['IK.n'], label='n')
plt.plot(runner.mon['ts'], runner.mon['INa.m'], label='m')
plt.plot(runner.mon['ts'], runner.mon['INa.h'], label='h')
plt.xlabel('t (ms)')
plt.legend()
plt.savefig("HH_channels.jpg")

plt.show()

  0%|          | 0/2000 [00:00<?, ?it/s]

<Figure size 432x288 with 1 Axes>

<Figure size 432x144 with 1 Axes>